# Notebook 1, VIX-Style 30-Day Variance Strip (SPX), t₀ = 2020-06-15

**Goal**  
From the SPX option chain at *t₀*, we build a VIX-style 30-day variance strip. The result is the fair 30-day variance and a list of OTM option legs that will feed Notebook 2 (delta-hedge) and Notebook 3 (VG model compare).

**Inputs.**
- `spx_option_chain_2020-06-15.csv`.  
- Strikes are scaled by `1/1000`. We set `r = 0`, `q = 0`.

**Methods.**  
Pick the two expiries in the VIX window (23–37 calendar days). For each, infer the forward from the near-ATM call/put spread, choose $K_0$ as the strike closest to the forward, and build OTM legs (puts for $K < K_0$, calls for $K \ge K_0$) with central strike spacing. Apply the standard VIX weights and the small adjustment term at $K_0$. Finally, interpolate the two per-expiry variances to exactly 30 days and report a VIX-style percentage.

**Outputs.**
- `01_variance_strike/strip_30d.csv` — per-expiry OTM legs and weights, each expiry includes a single audit row with type=B, not traded in Notebook 2.  
- `01_variance_strike/variance_strike_summary.csv` — near/next times, weights, per-expiry variances, 30-day variance, forwards, and $K_0$.

In [1]:
import math
from pathlib import Path
import numpy as np
import pandas as pd

BASE_DIR = Path.cwd()
OUT_DIR = BASE_DIR / "01_variance_strike"
OUT_DIR.mkdir(parents=True, exist_ok=True)
CHAIN_CSV = BASE_DIR / "spx_option_chain_2020-06-15.csv"

# Valuation date & VIX 30-day target
T0 = pd.Timestamp("2020-06-15")
CALENDAR_DAYS_TARGET = 30
DAYCOUNT_DEN = 365.0  # VIX uses calendar days / 365

# OptionMetrics-style strikes /1000
STRIKE_DIVISOR = 1000.0

r = 0.0

In [2]:
use_cols = ["date", "expiration", "cp_flag", "strike", "best_bid", "best_offer"]
chain = pd.read_csv(CHAIN_CSV, usecols=use_cols)

chain["date"] = pd.to_datetime(chain["date"])
chain["expiration"] = pd.to_datetime(chain["expiration"])

# fixed strike scaling
chain["strike"] = pd.to_numeric(chain["strike"], errors="coerce") / STRIKE_DIVISOR

# rename bid/ask, compute mid
chain["bid"] = pd.to_numeric(chain["best_bid"], errors="coerce")
chain["ask"] = pd.to_numeric(chain["best_offer"], errors="coerce")
chain = chain.drop(columns=["best_bid","best_offer"])
chain["mid"] = (chain["bid"] + chain["ask"]) / 2.0

# filter to t0
chain = chain[chain["date"].dt.date == T0.date()].copy()

In [3]:
exp = (
    chain[["expiration"]]
    .drop_duplicates()
    .assign(dte=lambda d: (d["expiration"].dt.date - T0.date()).apply(lambda x: x.days))
    .sort_values("dte")
)
window = exp.query("23 <= dte <= 37")

# prefer a true bracket around 30d, otherwise first two in window
near_row = window[window["dte"] <= 30].tail(1)
next_row = window[window["dte"] >= 30].head(1)
if near_row.empty or next_row.empty:
    near_row = window.iloc[[0]]
    next_row = window.iloc[[1]]

near_expiry = near_row["expiration"].iloc[0]
next_expiry = next_row["expiration"].iloc[0]

T_near = ((near_expiry.date() - T0.date()).days) / DAYCOUNT_DEN
T_next = ((next_expiry.date() - T0.date()).days) / DAYCOUNT_DEN
T_star = CALENDAR_DAYS_TARGET / DAYCOUNT_DEN

w_near = (T_next - T_star) / (T_next - T_near)
w_next = 1.0 - w_near

print(f"near={near_expiry.date()} T={T_near:.8f} | next={next_expiry.date()} T={T_next:.8f} | T*={T_star:.8f}")

near=2020-07-13 T=0.07671233 | next=2020-07-17 T=0.08767123 | T*=0.08219178


In [4]:
def compute_all(expiry: pd.Timestamp, T: float):
    df = chain[chain["expiration"] == expiry].copy()
    # pivot mids by strike
    piv = df.pivot_table(index="strike", columns="cp_flag", values="mid", aggfunc="first").dropna()
    # strike minimizing |C-P|
    k_star = float((piv["C"] - piv["P"]).abs().idxmin())
    C0 = float(piv.loc[k_star, "C"])
    P0 = float(piv.loc[k_star, "P"])
    # forward from parity (r=0)
    F = k_star + math.exp(r*T) * (C0 - P0)
    # find K0, strike closest to F (ties, pick the lower)
    strikes = piv.index.values
    diffs = np.abs(strikes - F)
    j = int(np.argmin(diffs))
    close_mask = np.isclose(diffs, diffs[j])
    if close_mask.sum() > 1:
        K0 = float(strikes[close_mask & (strikes <= F)].max())
    else:
        K0 = float(strikes[j])
    # OTM legs
    legs = df.loc[
        (df["cp_flag"].eq("P") & (df["strike"] < K0)) |
        (df["cp_flag"].eq("C") & (df["strike"] >= K0)),
        ["strike","cp_flag","mid"]
    ].sort_values("strike").copy()
    # strike spacing
    K = legs["strike"].values
    dK = np.empty_like(K)
    for i in range(len(K)):
        if i == 0:            dK[i] = K[i+1] - K[i]
        elif i == len(K)-1:   dK[i] = K[i]   - K[i-1]
        else:                 dK[i] = 0.5*(K[i+1] - K[i-1])
    legs["dK"] = dK
    legs["side"] = legs["cp_flag"]
    legs["type"] = "OTM"
    # VIX weight for each leg
    legs["weight"] = (2.0*math.exp(r*T) / T) * (legs["dK"] / (legs["strike"]**2))
    # per-expiry variance
    var_core = float((legs["weight"] * legs["mid"]).sum())
    # Boundary correction term B(T)
    B = (1.0/T) * ((F / K0 - 1.0)**2)
    # variance 
    sigma2 = var_core - B
    # append B row (audit only; excluded from hedging in Notebook 2)
    brow = pd.DataFrame({"strike":[K0],"cp_flag":["M"],"mid":[np.nan],"side":["M"],"dK":[np.nan],"weight":[B],"type":["B"]})
    legs = pd.concat([legs, brow], ignore_index=True)
    return float(sigma2), float(F), float(K0), legs

sigma2_near, F_near, K0_near, rows_near = compute_all(near_expiry, T_near)
sigma2_next, F_next, K0_next, rows_next = compute_all(next_expiry, T_next)

print(f"sigma2_near={sigma2_near:.10f} | sigma2_next={sigma2_next:.10f}")
print(f"F_near≈{F_near:.2f} K0_near={K0_near} | F_next≈{F_next:.2f} K0_next={K0_next}")

sigma2_near=0.1169832816 | sigma2_next=0.1346307533
F_near≈3058.80 K0_near=3060.0 | F_next≈3054.75 K0_next=3055.0


In [5]:
# VIX interpolation
sigma2_30d = w_near*sigma2_near + w_next*sigma2_next
VIX30_like_pct = 100.0 * math.sqrt(sigma2_30d)
print(f"VIX30_like_pct={VIX30_like_pct:.6f}%")

# Per-leg export (keep exact columns Notebook 2 expects; include single B row per expiry)
def prep_export(rows: pd.DataFrame, expiry: pd.Timestamp, T: float) -> pd.DataFrame:
    out = rows.copy()
    out["expiry"] = expiry.date().isoformat()
    out["T"] = float(T)
    out["r"] = float(r)
    return out[["expiry", "T", "r", "strike", "side", "type", "dK", "weight", "mid", "cp_flag"]]

export_near = prep_export(rows_near, near_expiry, T_near)
export_next = prep_export(rows_next, next_expiry, T_next)
strip_30d = pd.concat([export_near, export_next], ignore_index=True)
strip_path = OUT_DIR / "strip_30d.csv"
strip_30d.to_csv(strip_path, index=False)

summary = pd.DataFrame([{
    "t0": T0.date().isoformat(),
    "near": str(near_expiry.date()), "T_near": T_near,
    "next": str(next_expiry.date()), "T_next": T_next,
    "T_star": T_star, "w_near": w_near, "w_next": w_next,
    "sigma2_near": sigma2_near, "sigma2_next": sigma2_next, "sigma2_30d": sigma2_30d,
    "F_near": F_near, "F_next": F_next, "K0_near": K0_near, "K0_next": K0_next,
    "VIX30_like_pct": VIX30_like_pct
}])
summary_path = OUT_DIR / "variance_strike_summary.csv"
summary.to_csv(summary_path, index=False)

VIX30_like_pct=35.469285%
